In [ ]:
# FOR HDI: http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Information.php
# Human development classification
# HDI classifications are based on HDI fixed cutoff points, which are derived from the quartiles of dis
# tributions of the component indicators. The cutoffpoints are HDI of less than 0.550 for low human development,
#  0.550–0.699 for medium human development, 0.700–0.799 for high human development and 0.800 or greater 
#  for very high human development.
# Codes:
# 44206 - Total Population (millions)
# 137506 Human Development Index (HDI)

In [ ]:
import json
from turtle import back
import requests

In [ ]:
url_HDRO = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id=137506,44206/year=2019"
url_USGS = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

response_HDRO = requests.get(url_HDRO)
print(f"HDRO: {response_HDRO}")
response_USGS = requests.get(url_USGS)
print(f"USGS: {response_USGS}")

In [ ]:
data_HDRO = response_HDRO.json()
# print(json.dumps(data_HDRO, indent=4, sort_keys=True))
data_USGS = response_USGS.json()

In [ ]:
import pandas as pd

countryCodes_df = pd.read_csv("../../data/all.csv")

In [ ]:
countryCodes_df
clean_cc_df = countryCodes_df.copy()[["name", "alpha-2", "alpha-3"]]
clean_cc_df

df_HDRO = pd.read_json(url_HDRO)
df_HDRO = df_HDRO.reset_index()
df_HDRO

In [ ]:
clean_HDRO_df = pd.DataFrame(columns=['CountryName', 'CountryCode', 'HDI', "TotalPopulation"]) 
# clean_HDRO_df
i=1
for index, country in df_HDRO.iterrows():
    try:
        value_totalPopulation = float(country["indicator_value"]['44206']['2019'])
        value_HDI = float(country["indicator_value"]['137506']['2019'])
        clean_HDRO_df = clean_HDRO_df.append([{"CountryName": country["country_name"], "CountryCode": country["index"], "HDI": value_HDI, "TotalPopulation": value_totalPopulation}], ignore_index=True)
    except:
        print(country["index"], country["indicator_value"])

clean_HDRO_df


In [ ]:
clean_HDRO_df = clean_HDRO_df.merge(clean_cc_df, left_on="CountryCode", right_on="alpha-3", how="inner")
clean_HDRO_df =clean_HDRO_df.drop(["CountryCode"], axis=1)


In [ ]:
clean_HDRO_df

In [ ]:
for index, row in clean_HDRO_df.iterrows():
    print(row)

In [ ]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import module to create relationship patterns
from sqlalchemy.orm import relationship

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Table, Column, Integer, String, Float, BigInteger, ForeignKey

In [ ]:
# Create classes:
# Countries & Earthquakes

class Countries(Base):  #Parent
    __tablename__ = 'countries'
    # id = Column(Integer, primary_key=True)
    CountryCode = Column(String(255), primary_key=True)
    CountryName = Column(String(255))
    Population = Column(Float)  # Millions
    HDIndex = Column(Float) # Index
    earthquake = relationship("Earthquakes")

class Earthquakes(Base):    #Child
    __tablename__ = 'earthquakes'
    # id = Column(Integer, primary_key=True)
    EarthquakeID = Column(String(255), primary_key=True)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Depth = Column(Float)
    DateTime = Column(BigInteger)
    # CountryCode = Column(String(255))   # Foreign Key?
    country_id = Column(String(255), ForeignKey('countries.CountryCode'))
    # parent = relationship("Countries", back_populates="children")


# print(data_USGS["features"])

In [ ]:
import reverse_geocoder as rg
from time import sleep
from tqdm.auto import tqdm

In [ ]:
# Create Database Connection
# ---------------------------
# Establish Connection
engine = create_engine("sqlite:///project2.sqlite")
conn = engine.connect()

# Create Countries and Earthquakes tables within the database
Base.metadata.create_all(conn)

# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
for index, country in tqdm(clean_HDRO_df.iterrows()):
    sleep(0.1)
    try:
        new_country = Countries(CountryCode=country["alpha-2"],CountryName=country["CountryName"],Population=country["TotalPopulation"],HDIndex=country["HDI"])
        session.add(new_country)
        session.commit()
    except:
        print(country["alpha-2"], country["CountryName"], country["TotalPopulation"], country["HDI"])


In [ ]:
i=1
# Create specific instances of the Countries and Earthquakes classes
for earthquake in tqdm(data_USGS["features"]):
    sleep(0.1)

    if i<=10:
        # print(earthquake["id"], earthquake["geometry"]["coordinates"][0], earthquake["geometry"]["coordinates"][1], earthquake["geometry"]["coordinates"][2], earthquake["properties"]["time"])
        lon = earthquake["geometry"]["coordinates"][0]
        lat = earthquake["geometry"]["coordinates"][1]
        depth1 = earthquake["geometry"]["coordinates"][2]
        ctry_code = rg.search((lat,lon))[0]["cc"]
        new_earthquake = Earthquakes(EarthquakeID=earthquake["id"], Longitude=lon, Latitude=lat, Depth=depth1, DateTime=earthquake["properties"]["time"], country_id=ctry_code)

        print(f"{i}, {earthquake['id']}, {lon}, {lat}, {depth1}, {earthquake['properties']['time']}, {ctry_code}")
        # Add new_earthquake to the current session
        session.add(new_earthquake)
        # Add new_country to the current session

        # Commit objects to the database
        session.commit()
        # print(i)
        i = i+1
    else:
        break


conn.close()
engine.dispose()




earthquake_list = session.query(Earthquakes)
for earthquake in earthquake_list:
    print(earthquake.Longitude)